In [ ]:
%pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.1 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.11-py3-none-any.whl size=5595978 sha256=bda439628a673de9182fa09e904b43ad01d50cfe42f0b09dc4006983e69c0f76
  Stored in directory: /root/.cache/pip/wheels/f4/ad/93/a7042fe12668827574927ade9deec7f29aad2a1001b1501882
Successfully built pretty-midi


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/MyDrive/bitmidi_temp'


In [ ]:
import os
from pathlib import Path
import numpy as np
import pypianoroll
import pretty_midi
import warnings


INPUT_DIR = "/content/drive/MyDrive/bitmidi_temp"

# On va créer un dossier 'processed_musegan' dans ton Drive
OUTPUT_DIR = "/content/drive/MyDrive/processed_musegan"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = os.path.join(OUTPUT_DIR, "musegan_training_data_part")

RECURSIVE = True
START_FILE = 0      # On commence du début
END_FILE = None     # Jusqu'à la fin
PLATFORM_NAME = "Colab_Drive"

# --- PARAMÈTRES (Inchangés) ---
FAMILY_NAMES = ["drum", "bass", "guitar", "string", "piano"]
FAMILY_THRESHOLDS = [(1, 12), (1, 48), (1, 78), (1, 78), (1, 78)]
RESOLUTION = 24
SAVE_BATCH_SIZE = 5000 # J'ai réduit un peu pour être sûr avec le Drive (plus lent)

warnings.filterwarnings('ignore', category=RuntimeWarning)

# --- FONCTIONS ---
def check_which_family(instrument):
    program = instrument.program; is_drum = instrument.is_drum
    def is_piano(): return not is_drum and ((0 <= program <= 7) or (16 <= program <= 23))
    def is_guitar(): return 24 <= program <= 31
    def is_bass(): return 32 <= program <= 39
    def is_string(): return 40 <= program <= 51
    return np.array([is_drum, is_bass(), is_guitar(), is_string(), is_piano()])

def segment_quality(pianoroll_segment, threshold_pitch, threshold_beats):
    pitch_sum = np.sum(np.sum(pianoroll_segment, axis=0) > 0)
    beat_sum = np.sum(np.sum(pianoroll_segment, axis=1) > 0)
    return ((pitch_sum >= threshold_pitch) and (beat_sum >= threshold_beats)), (pitch_sum, beat_sum)

def save_batch(segments, part_number):
    if not segments:
        return
    output_filename = f"{OUTPUT_PREFIX}_{part_number}.npz"
    print("-" * 30)
    print(f"💾 Sauvegarde du lot n°{part_number} ({len(segments)} segments)...")
    print(f"📍 Fichier : {output_filename}")
    np.savez_compressed(output_filename, *segments)
    print("✅ Lot sauvegardé !")
    print("-" * 30)

def main():
    print("🚀 Début du traitement sur ton Dataset BitMidi...")
    input_path = Path(INPUT_DIR)

    if not input_path.exists():
        print(f"❌ ERREUR : Le dossier n'existe pas : {input_path}")
        print("Vérifie que tu as bien monté le Drive (Étape 1).")
        return

    print("📂 Recherche des fichiers MIDI (.mid et .MID)...")
    # Modification pour trouver .mid ET .MID
    files_lower = list(input_path.rglob("*.mid"))
    files_upper = list(input_path.rglob("*.MID"))
    # On combine et on enlève les doublons
    all_filenames = sorted(list(set(files_lower + files_upper)))

    print(f"🎵 {len(all_filenames)} fichiers MIDI trouvés au total.")

    if len(all_filenames) == 0:
        print("⚠️ Aucun fichier trouvé. Vérifie le chemin du dossier.")
        return

    files_to_process = all_filenames[START_FILE:END_FILE]
    batch_of_segments = []
    part_number = 1
    total_files = len(files_to_process)

    print(f"Traitement de {total_files} fichiers en cours...")

    for i, filename in enumerate(files_to_process):
        # Affichage de progression tous les 100 fichiers
        if (i + 1) % 100 == 0:
            print(f"⏳ Progression : {i + 1} / {total_files} fichiers analysés...")

        try:
            pm = pretty_midi.PrettyMIDI(str(filename))
            downbeats = pm.get_downbeats()
            if len(downbeats) < 5: continue

            multitrack = pypianoroll.from_pretty_midi(pm, resolution=RESOLUTION)
            num_bars = len(downbeats) - 1

            for bar_idx in range(0, num_bars - 4, 2):
                start_tick = int(downbeats[bar_idx] * RESOLUTION)
                end_tick = int(downbeats[bar_idx + 4] * RESOLUTION)

                segment_tracks = {name: None for name in FAMILY_NAMES}
                best_scores = {name: -1 for name in FAMILY_NAMES}

                for track in multitrack.tracks:
                    family_map = check_which_family(track)
                    in_family_idx = np.where(family_map)[0]
                    if not len(in_family_idx): continue

                    family_name = FAMILY_NAMES[in_family_idx[0]]
                    family_thresholds = FAMILY_THRESHOLDS[in_family_idx[0]]

                    if track.pianoroll.shape[0] < end_tick: continue

                    pianoroll_segment = track.pianoroll[start_tick:end_tick]
                    is_ok, score = segment_quality(pianoroll_segment, family_thresholds[0], family_thresholds[1])

                    if is_ok and sum(score) > best_scores[family_name]:
                        best_scores[family_name] = sum(score)
                        segment_tracks[family_name] = pianoroll_segment

                valid_instruments_count = sum(1 for track in segment_tracks.values() if track is not None)
                if valid_instruments_count >= 2:
                    final_segment = []
                    TARGET_LENGTH = 4 * RESOLUTION

                    for name in FAMILY_NAMES:
                        if segment_tracks[name] is not None:
                            pianoroll = segment_tracks[name]
                            padded_pianoroll = np.zeros((TARGET_LENGTH, 128), dtype=bool)
                            actual_length = min(TARGET_LENGTH, pianoroll.shape[0])
                            padded_pianoroll[:actual_length, :] = pianoroll[:actual_length, :] > 0
                            final_segment.append(padded_pianoroll[:, :, np.newaxis])
                        else:
                            final_segment.append(np.zeros((TARGET_LENGTH, 128, 1), dtype=bool))

                    compiled_segment = np.concatenate(final_segment, axis=2)
                    batch_of_segments.append(compiled_segment)

                    # Sauvegarde intermédiaire
                    if len(batch_of_segments) >= SAVE_BATCH_SIZE:
                        save_batch(batch_of_segments, part_number)
                        part_number += 1
                        batch_of_segments.clear()
        except Exception as e:
            # On ignore les erreurs de fichiers corrompus pour ne pas stopper le script
            continue

    # Sauvegarde finale
    print("🏁 Fin du scan. Sauvegarde des derniers segments...")
    save_batch(batch_of_segments, part_number)
    print(f"🎉 Terminé ! Tes données sont dans : {OUTPUT_DIR}")

if __name__ == "__main__":
    main()


🚀 Début du traitement sur ton Dataset BitMidi...
📂 Recherche des fichiers MIDI (.mid et .MID)...
🎵 27640 fichiers MIDI trouvés au total.
Traitement de 27640 fichiers en cours...
⏳ Progression : 100 / 27640 fichiers analysés...
⏳ Progression : 200 / 27640 fichiers analysés...
⏳ Progression : 300 / 27640 fichiers analysés...
------------------------------
💾 Sauvegarde du lot n°1 (5000 segments)...
📍 Fichier : /content/drive/MyDrive/processed_musegan/musegan_training_data_part_1.npz
✅ Lot sauvegardé !
------------------------------
⏳ Progression : 400 / 27640 fichiers analysés...
⏳ Progression : 500 / 27640 fichiers analysés...
------------------------------
💾 Sauvegarde du lot n°2 (5000 segments)...
📍 Fichier : /content/drive/MyDrive/processed_musegan/musegan_training_data_part_2.npz
✅ Lot sauvegardé !
------------------------------
⏳ Progression : 600 / 27640 fichiers analysés...
⏳ Progression : 700 / 27640 fichiers analysés...
⏳ Progression : 800 / 27640 fichiers analysés...
----------